In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
# in meters
DISTANCE_FROM_STATION = 2000

In [3]:
areas = gpd.read_file("data/Boundaries - Neighborhoods.geojson")
block_groups = gpd.read_file("data/Boundaries - Census Blocks - 2010.geojson")

In [4]:
areas.head()

,pri_neigh,sec_neigh,shape_area,shape_len,geometry
0,Grand Boulevard,BRONZEVILLE,48492503.1554,28196.837157,"MULTIPOLYGON (((-87.60671 41.81681, -87.60670 ..."
1,Printers Row,PRINTERS ROW,2162137.97139,6864.247156,"MULTIPOLYGON (((-87.62761 41.87437, -87.62760 ..."
2,United Center,UNITED CENTER,32520512.7053,23101.363745,"MULTIPOLYGON (((-87.66707 41.88885, -87.66707 ..."
3,Sheffield & DePaul,SHEFFIELD & DEPAUL,10482592.2987,13227.049745,"MULTIPOLYGON (((-87.65833 41.92166, -87.65835 ..."
4,Humboldt Park,HUMBOLDT PARK,125010425.593,46126.751351,"MULTIPOLYGON (((-87.74060 41.88782, -87.74060 ..."


In [5]:
block_groups.head()

,statefp10,name10,blockce10,tract_bloc,geoid10,tractce10,countyfp10,geometry
0,17,Block 2010,2010,6903002010,170316903002010,690300,031,"MULTIPOLYGON (((-87.62907 41.76909, -87.62905 ..."
1,17,Block 3007,3007,6809003007,170316809003007,680900,031,"MULTIPOLYGON (((-87.63412 41.77447, -87.63410 ..."
2,17,Block 3013,3013,6809003013,170316809003013,680900,031,"MULTIPOLYGON (((-87.63485 41.77263, -87.63522 ..."
3,17,Block 4019,4019,2909004019,170312909004019,290900,031,"MULTIPOLYGON (((-87.73841 41.85913, -87.73842 ..."
4,17,Block 4016,4016,2925004016,170312925004016,292500,031,"MULTIPOLYGON (((-87.73217 41.85476, -87.73226 ..."


In [6]:
block_groups.shape

(46357, 8)

In [7]:
block_groups_areas = block_groups.copy()
block_groups_areas["blockgroup"] = block_groups_areas["geoid10"].str[0:12]
block_groups_areas = block_groups_areas[["geometry", "blockgroup"]]

block_groups_areas = gpd.sjoin(block_groups_areas, areas, how="left", op='intersects')
block_groups_areas = block_groups_areas.dissolve(by='blockgroup', aggfunc='first')

block_groups_areas = block_groups_areas[["geometry", "pri_neigh"]].reset_index()
block_groups_areas.head()

,blockgroup,geometry,pri_neigh
0,170310101001,"POLYGON ((-87.67009 42.02115, -87.67047 42.021...",Rogers Park
1,170310101002,"POLYGON ((-87.66950 42.01936, -87.66963 42.019...",Rogers Park
2,170310101003,"POLYGON ((-87.66681 42.01924, -87.66780 42.019...",Rogers Park
3,170310102011,"POLYGON ((-87.68234 42.01250, -87.68268 42.012...",Rogers Park
4,170310102012,"POLYGON ((-87.67972 42.01392, -87.68003 42.013...",Rogers Park


In [8]:
block_groups_areas.crs

{'init': 'epsg:4326'}

In [9]:
stations = (
    pd.read_csv(
        "data/CTA_L_Stops.csv",
        usecols=["STATION_DESCRIPTIVE_NAME", "MAP_ID", "Location"],
    )
    .drop_duplicates("MAP_ID")
    .rename(
        columns={"STATION_DESCRIPTIVE_NAME": "station_name", "MAP_ID": "station_id"}
    )
)
stations_points = []
for location in stations["Location"]:
    lon, lat = [float(coord) for coord in location.strip("()").split(", ")]
    stations_points.append(Point(lat, lon))
stations = gpd.GeoDataFrame(stations, geometry=stations_points)[
    ["station_id", "station_name", "geometry"]
]
stations.crs = {'init': 'epsg:4326'}
stations.head()

,station_id,station_name,geometry
0,40830,18th (Pink Line),POINT (-87.66915 41.85791)
2,40120,35th/Archer (Orange Line),POINT (-87.68062 41.82935)
4,41120,35th-Bronzeville-IIT (Green Line),POINT (-87.62583 41.83168)
6,41270,43rd (Green Line),POINT (-87.61902 41.81646)
8,40130,51st (Green Line),POINT (-87.61849 41.80209)


In [10]:
stations.shape

(143, 3)

In [11]:
stations = stations.to_crs({"init": "epsg:26971"})
block_groups_areas = block_groups_areas.to_crs({"init": "epsg:26971"})

In [12]:
stations_blockgroups = []
for station in stations.itertuples():
    for area in block_groups_areas.itertuples():
        # distances are in meters, so we look for blockgroups that are at most 1km away
        dist = station.geometry.hausdorff_distance(area.geometry)
        if dist <= DISTANCE_FROM_STATION:
            stations_blockgroups.append(
                {
                    "station_id": station.station_id,
                    "station_name": station.station_name,
                    "blockgroup": area.blockgroup
                }
            )
stations_blockgroups = pd.DataFrame(stations_blockgroups)
stations_blockgroups.head()

,station_id,station_name,blockgroup
0,40830,18th (Pink Line),170312831001
1,40830,18th (Pink Line),170312831002
2,40830,18th (Pink Line),170312832001
3,40830,18th (Pink Line),170312838001
4,40830,18th (Pink Line),170312838002


In [13]:
stations_blockgroups["blockgroup"].unique().shape

(1340,)

In [14]:
ridership = pd.read_csv(
    "data/CTA_L_Ridership_Monthly_Day-Type.csv",
    usecols=["station_id", "month_beginning", "monthtotal"]
).rename(columns={"monthtotal": "train_rides"})
stations_ridership = pd.merge(stations_blockgroups, ridership, on="station_id")
stations_ridership.head()

,station_id,station_name,blockgroup,month_beginning,train_rides
0,40830,18th (Pink Line),170312831001,01/01/2001,21301
1,40830,18th (Pink Line),170312831001,02/01/2001,19955
2,40830,18th (Pink Line),170312831001,03/01/2001,21998
3,40830,18th (Pink Line),170312831001,04/01/2001,21297
4,40830,18th (Pink Line),170312831001,05/01/2001,22675


In [15]:
stations_ridership.shape

(1208571, 5)

In [16]:
blockgroup_ridership = stations_ridership.groupby(
    ["blockgroup", "month_beginning"],
    as_index=False
)[["blockgroup", "month_beginning", "train_rides"]].sum()
blockgroup_ridership.head()

,blockgroup,month_beginning,train_rides
0,170310101001,01/01/2001,377399
1,170310101001,01/01/2002,356035
2,170310101001,01/01/2003,340125
3,170310101001,01/01/2004,324526
4,170310101001,01/01/2005,325312


In [17]:
blockgroup_ridership["blockgroup"].unique().shape

(1340,)

In [18]:
blockgroup_ridership.to_csv("data/CTA_L_Ridership_Monthly_by_Blockgroup.csv", index=False)

In [19]:
# TODO; this takes a while
# divvy = gpd.read_file("data/Divvy_Trips.csv", parse_dates=["START TIME"])
# divvy.head()